In this Tutorial we will explore how to work with columnar data in HoloViews. The majority of data we work with on a daily basis can be described in columns from spreadsheets to databases. HoloViews defines an extensible system of interfaces to load, operate on and visualize this kind of data.

By default HoloViews will use one of three interfaces to operate on the data:

* A pure Python dictionary containing each column.
* A purely numpy based format for numeric data.
* Pandas DataFrames

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
hv.notebook_extension()

# Simple Columns

Usually when working with data we have one or more independent variables, taking the form of categories, discrete samples or bins, these are what we refer to as key dimensions or kdims for short in HoloViews. The observed or dependent variables on the other hand are referred to as value dimensions (vdims). The simplest form of a Columns object is therefore a column 'x' and a column 'y' corresponding to the key dimensions and value dimensions respectively. A simple visual representation of this data is a Table, however there are many differ ways to represent this data.

In [ ]:
xs = range(10)
ys = np.linspace(0, 1, 10)
table = hv.Table((xs, ys), kdims=['x'], vdims=['y'])

However this data has many more meaningful visual representations, therefore the first important concept is that Columns objects are interchangeable as long as their dimensionality allows it, meaning that you can easily cast between them.

In [ ]:
hv.Scatter(table) + hv.Curve(table) + hv.Bars(table)

The choice of an appropriate Element depends on the dimensionality of the data, here's a rough listing of the dimensionality of different Element types:

* 0D: BoxWhisker, Spikes, Distribution*, 
* 1D: Scatter, Curve, ErrorBars, Spread, Bars, BoxWhisker, Regression*
* 2D: Points, HeatMap, Bars, BoxWhisker, Bivariate*
* 3D: Scatter3D, Trisurface, VectorField, BoxWhisker, Bars

\* - requires Seaborn

This only represents the dimensionality of the actual sampling of each Element. Additionally an Element can have any number of value dimensions, which may be mapped onto various attributes of a plot such as the color, size, angle of the plot elements, for a reference of how to use these various Element types see the [Elements Tutorial](Elements).

## Data types and Constructors

As discussed above Columns provide an extensible interface to store and operate on data in different formats. All interfaces support a number of standard constructors.

#### Storage formats

Columns types can be constructed using one of three supported formats, (a) a dictionary of columns, (b) an NxD array with N rows and D columns, or (c) pandas dataframes:

In [ ]:
print(repr(hv.Scatter({'x': xs, 'y': ys}) +
           hv.Scatter(np.column_stack([xs, ys])) +
           hv.Scatter(pd.DataFrame({'x': xs, 'y': ys}))))

#### Literals

In addition to the main storage format, Columns support three Python literal formats, (a) An iterator of y-values, (b) a tuple of columns, and (c) an iterator of row tuples.

In [ ]:
print(repr(hv.Scatter(ys) + hv.Scatter((xs, ys)) + hv.Scatter(zip(xs, ys))))

By default Columns will try to construct a simple array, falling back to either pandas dataframes (if available) or the dictionary based format if the data is not purely numeric. Additionally the interfaces will try to maintain their type, numpy arrays and pandas DataFrame will therefore always be parsed by the array and dataframe interfaces first respectively.

In [ ]:
df = pd.DataFrame({'x': xs, 'y': ys, 'z': ys*2})
print type(hv.Scatter(df).data)

Columns will attempt to parse the supplied data falling back to each consecutive interface if the previous could not interpret the data. The default list of fallbacks and simultaneously the list of allowed datatypes is:

In [ ]:
hv.Columns.datatype

To select a particular storage format supply one or more allowed datatypes:

In [ ]:
print(type(hv.Scatter((xs, ys), datatype=['array']).data))
print(type(hv.Scatter((xs, ys), datatype=['dictionary']).data))
print(type(hv.Scatter((xs, ys), datatype=['dataframe']).data))

#### Sharing Data

Since the formats with labelled columns do not require any specific order the Elements can effectively become views into the data. By specifying different key and value dimensions many Elements can share the same data source.

In [ ]:
overlay = hv.Scatter(df, kdims='x', vdims='y') * hv.Scatter(df, kdims='x', vdims='z')
overlay

We can quickly confirm that the data is actually shared:

In [ ]:
overlay.Scatter.I.data is overlay.Scatter.II.data

This is much more efficient than creating copies of the data for each Element and allows for some advanced features like linked brushing in the [Bokeh backend](Bokeh_Backend).

#### Converting to raw data

Column types make it easy to export the data to the three basic formats, arrays, dataframes and a dictionary of columns.

###### Array

In [ ]:
table.array()

###### Pandas DataFrame

In [ ]:
table.dframe().head()

###### Columns dictionary

In [ ]:
table.columns()

## Applying operations to the data

#### Basic Operations

Columns can be sorted by their dimensions using the ``sort`` method, by default it will sort by the key dimensions but by supplying the dimension it is possible to sort by any dimension(s):

In [ ]:
bars = hv.Bars((['C', 'A', 'B', 'D'], [2, 7, 3, 4]))
bars + bars.sort() + bars.sort(['y'])

#### Working with categorical or grouped data

Data is often grouped in various ways and the Columns interface provides various means to easily compare between groups and apply statistical aggregates. We'll start by generating some synthetic data with two groups along the x-axis and 4 groups along the y axis.

In [ ]:
%%opts Table [aspect=2 fig_size=200]
np.random.seed(42)
n = np.arange(1000)
xs = np.repeat(range(2), 500)
ys = n%4
zs = np.random.randn(1000)
table = hv.Table((xs, ys, zs), kdims=['x', 'y'], vdims=['z'])
table

Since there are repeat observations of the same x- and y-values we have to reduce the data before we display it or use a datatype that supports plotting distributions in this way. The ``BoxWhisker`` type allows doing exactly that:

In [ ]:
%%opts BoxWhisker [aspect=2 fig_size=200 bgcolor='w']
hv.BoxWhisker(table)

### Aggregating/Reducing dimensions

Most types require the data to be non-duplicated before being displayed for this purpose HoloViews makes it easy to ``aggregate`` and ``reduce`` the data. These two operations are simple inverses of each other, aggregate computes a statistic for each group in the supplied dimensions, while reduce aggregates over all the groups except the supplied dimensions. Supplying only a function and no dimensions will simply aggregate or reduce all available key dimensions.

In [ ]:
%%opts Bars [show_legend=False] {+axiswise}
hv.Bars(table).aggregate(function=np.mean) + hv.Bars(table).reduce(x=np.mean)

(**A**) aggregates over both the x and y dimension, computing the mean for each x/y group, while (**B**) reduces the x dimension leaving just the mean for each group along y.

##### Collapsing multi Columns Elements

When multiple observations are broken out into a HoloMap they can easily be combined using the ``collapse`` method. Here we create a number of Curves with increasingly larger y-values. By collapsing them with a ``function`` and a ``spreadfn`` we can compute the mean curve with a confidence interval. We simply cast the collapsed ``Curve`` to a ``Spread`` and ``Curve`` Element to visualize them.

In [ ]:
hmap = hv.HoloMap({i: hv.Curve(np.arange(10)*i) for i in range(10)})
collapsed = hmap.collapse(function=np.mean, spreadfn=np.std)
hv.Spread(collapsed) * hv.Curve(collapsed) + collapsed.table()

## Working with complex data

In the last section we only scratched the surface of what the Columns interface can do, when it really comes into its own is when working with high dimensional datasets. We'll load a dataset of some macro-economic indicators for a OECD countries from 1964-1990 from the HoloViews website.

In [ ]:
macro_df = pd.read_csv('http://ioam.github.com/holoviews/Tutorials/macro.csv', '\t')

dimensions = {'unem': 'Unemployment',
              'capmob': 'Capital Mobility',
              'gdp': 'GDP Growth', 'trade': 'Trade',
              'year': 'Year', 'country': 'Country'}

macro_df = macro_df.rename(columns=dimensions)

We'll also take this opportunity to set options for all the following plots.

In [ ]:
%output dpi=100
options = hv.Store.options()
opts = hv.Options('plot', aspect=2, fig_size=250, show_frame=False, show_grid=True, legend_position='right')
options.NdOverlay = opts
options.Overlay = opts

###### Loading the data

As we saw above we can supply the dataframe to any Columns type. When dealing with so many dimensions it would be cumbersome to supply all the dimensions explicitly, therefore Columns can easily infer the dimensions from a dataframe. We simply supply the ``kdims`` and it will infer that all other numeric dimensions should be treated as value dimensions (``vdims``).

In [ ]:
macro = hv.Table(macro_df, kdims=['Year', 'Country'])

To get an overview of the data we'll quickly sort it and then view the data for one year.

In [ ]:
%%opts Table [aspect=1.5 fig_size=300]
macro = macro.sort()
macro[1988]

Above we looked at converting a Table to simple Element types, however HoloViews also provides powerful container objects to explore high-dimensional data, currently these are [HoloMap](http://ioam.github.io/holoviews/Tutorials/Containers.html#HoloMap), [NdOverlay](http://ioam.github.io/holoviews/Tutorials/Containers.html#NdOverlay), [NdLayout](http://ioam.github.io/holoviews/Tutorials/Containers.html#NdLayout) and [GridSpace](http://ioam.github.io/holoviews/Tutorials/Containers.html#Layout). HoloMaps provide the basic conversion type from which you can conveniently convert to the other container types using the ``.overlay``, ``.layout`` and ``.grid`` methods. This way we can easily create an overlay of GDP Growth curves by year for each country. Here 'Year' is a key dimension and 'GDP Growth' a value dimension. Additionally we are left with the 'Country' dimension, which we then overlay calling the ``.overlay method``.

In [ ]:
%%opts Curve (color=Palette('Set3'))
gdp_curves = macro.to.curve('Year', 'GDP Growth')
gdp_curves.overlay('Country')

Now that we've extracted the gdp_curves we can apply some operations to them. As in the simpler example above we will ``collapse`` the HoloMap of Curves using a number of functions to visualize the distribution of GDP Growth rates over time. First we find the mean curve with np.std as the ``spreadfn`` and cast the result to a ``Spread`` type, then we compute the min, mean and max curve in the same way and put them inside an Overlay.

In [ ]:
%%opts Overlay [bgcolor='w' legend_position='top_right'] Curve (color='k' linewidth=1) Spread (color='gray' alpha=0.2)
hv.Spread(gdp_curves.collapse('Country', np.mean, np.std), label='std') *\
hv.Overlay([gdp_curves.collapse('Country', fn).relabel(name)(style=dict(linestyle=ls))
            for name, fn, ls in [('max', np.max, '--'), ('mean', np.mean, '-'), ('min', np.min, '--')]])

Many HoloViews Element types support multiple kdims, including HeatMaps, Points, Scatter, Scatter3D, and Bars. Bars in particular allows you to lay out your data in groups, categories and stacks. By supplying the index of that dimension as a plotting option you can choose to lay out your data as groups of bars, categories in each group and stacks. Here we choose to lay out the trade surplus of each country with groups for each year, no categories, and stacked by country. Finally we choose to color the Bars for each item in the stack.

In [ ]:
%opts Bars [bgcolor='w' aspect=3 figure_size=450 show_frame=False]

In [ ]:
%%opts Bars [category_index=2 stack_index=0 group_index=1 legend_position='top' legend_cols=7 color_by=['stack']] (color=Palette('Dark2'))
macro.to.bars(['Country', 'Year'], 'Trade', [])

Using the .select method we can pull out the data for just a few countries and specific years. We can also make more advanced use the Palettes.

Palettes can customized by selecting only a subrange of the underlying cmap to draw the colors from. The Palette draws samples from the colormap using the supplied sample_fn, which by default just draws linear samples but may be overriden with any function that draws samples in the supplied ranges. By slicing the Set1 colormap we draw colors only from the upper half of the palette and then reverse it.

In [ ]:
%%opts Bars [padding=0.02 color_by=['group']] (alpha=0.6, color=Palette('Set1', reverse=True)[0.:.2])
countries = {'Belgium', 'Netherlands', 'Sweden', 'Norway'}
macro.to.bars(['Country', 'Year'], 'Unemployment').select(Year=(1978, 1985), Country=countries)

Many HoloViews Elements support multiple key and value dimensions. A HeatMap may be indexed by two kdims, so we can visualize each of the economic indicators by year and country in a Layout. Layouts are useful for heterogeneous data you want to lay out next to each other.

Before we display the Layout let's apply some styling, we'll suppress the value labels applied to a HeatMap by default and substitute it for a colorbar. Additionally we up the number of xticks that are drawn and rotate them by 90 degrees to avoid overlapping. Flipping the y-axis ensures that the countries appear in alphabetical order. Finally we reduce some of the margins of the Layout and increase the size.

In [ ]:
%opts HeatMap [show_values=False xticks=40 xrotation=90 aspect=1.2 invert_yaxis=True]
%opts Layout [figure_size=120 aspect_weight=0.2]

In [ ]:
hv.Layout([macro.to.heatmap(['Year', 'Country'], value).relabel(value)
           for value in macro.data.columns[2:]]).cols(2)

Another way of combining heterogeneous data dimensions is to map them to a multi-dimensional plot type. Scatter Elements for example support multiple ``vdims``, which may be mapped onto the color and size of the drawn points in addition to the y-axis position. 

As for the Curves above we supply 'Year' as the sole key dimension and rely on the Table to automatically convert the Country to a map dimension, which we'll overlay. However this time we select both GDP Growth and Unemployment but to be plotted as points. To get a sensible chart, we adjust the scaling_factor for the points to get a reasonable distribution in sizes and apply a categorical Palette so we can distinguish each country.

In [ ]:
%%opts Scatter [scaling_factor=1.4] (color=Palette('Set3') edgecolors='k')
gdp_unem_scatter = macro.to.scatter('Year', ['GDP Growth', 'Unemployment'])
gdp_unem_scatter.overlay('Country')

In this way we can plot any dimension against any other dimension very easily allowing us to iterate through different ways of revealing relationships in the dataset.

In [ ]:
%%opts NdOverlay [legend_cols=2] Scatter [size_index=1 scaling_factor=1.3] (color=Palette('Blues'))
macro.to.scatter('GDP Growth', 'Unemployment', ['Year']).overlay()

This view for example immediately highlights the high unemployment rates of the 80s.

Since all HoloViews Elements are composable we can generate complex figures just by applying the * operator. We'll simply reuse the GDP curves we generated earlier, combine them with the scatter points, which indicate the unemployment rate by size and annotate the data with some descriptions of what happened economically in these years.

In [ ]:
%%opts Curve (color='k') Scatter [color_index=2 size_index=2 scaling_factor=1.4] (cmap='Blues' edgecolors='k')
macro_overlay = gdp_curves * gdp_unem_scatter
annotations = hv.Arrow(1973, 8, 'Oil Crisis', 'v') * hv.Arrow(1975, 6, 'Stagflation', 'v') *\
hv.Arrow(1979, 8, 'Energy Crisis', 'v') * hv.Arrow(1981.9, 5, 'Early Eighties\n Recession', 'v')
macro_overlay * annotations

Since we didn't map the country to some other container type, we get a widget allowing us to view the plot separately for each country, reducing the forest of curves we encountered before to manageable chunks. 

While looking at the plots individually like this allows us to study trends for each country, we may want to lay outa subset of the countries side by side. We can easily achieve this by selecting the countries we want to view and and then applying the ``.layout`` method. We'll also want to restore the aspect so the plots compose nicely.

In [ ]:
%opts Overlay [aspect=1]

In [ ]:
%%opts NdLayout [figure_size=100] Scatter [color_index=2] (cmap='Reds')
countries = {'United States', 'Canada', 'United Kingdom'}
(gdp_curves * gdp_unem_scatter).select(Country=countries).layout('Country')

Finally let's combine some plots for each country into a Layout, giving us a quick overview of each economic indicator for each country:

In [ ]:
%%opts Layout [fig_size=100] Scatter [color_index=2] (cmap='Reds')
(macro_overlay.relabel('GDP Growth', depth=1) +\
macro.to.curve('Year', 'Unemployment', ['Country'], group='Unemployment',) +\
macro.to.curve('Year', 'Trade', ['Country'], group='Trade') +\
macro.to.scatter('GDP Growth', 'Unemployment', ['Country'])).cols(2)